In [1]:
import pandas as pd

In [10]:
excel_file_path=r"..\data\ODIR-5K_Training_Annotations(Updated)_V2.xlsx"
annoted_data=pd.read_excel(excel_file_path)

In [11]:
annoted_data

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,4686,63,Male,4686_left.jpg,4686_right.jpg,severe nonproliferative retinopathy,proliferative diabetic retinopathy,0,1,0,0,0,0,0,0
3496,4688,42,Male,4688_left.jpg,4688_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
3497,4689,54,Male,4689_left.jpg,4689_right.jpg,mild nonproliferative retinopathy,normal fundus,0,1,0,0,0,0,0,0
3498,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0


In [18]:

data = pd.read_csv("..\data\data.csv")

In [34]:
#unique_left_labels = pd.unique(data['Left-Diagnostic Keywords']).tolist()
keywords = data['Left-Diagnostic Keywords'].str.split(',')
print(keywords)
unique_left_keywords = pd.unique(keywords.explode()).tolist()

print(len(unique_left_keywords))
#unique_right_labels = pd.unique(data['Right-Diagnostic Keywords']).tolist()

225


In [20]:
len(unique_left_labels)

227

In [25]:
print(unique_left_labels)

['cataract', 'normal fundus', 'laser spot，moderate non proliferative retinopathy', 'macular epiretinal membrane', 'moderate non proliferative retinopathy', 'drusen', 'epiretinal membrane', 'moderate non proliferative retinopathy，hypertensive retinopathy', 'retinal pigmentation', 'pathological myopia', 'mild nonproliferative retinopathy', 'rhegmatogenous retinal detachment', 'moderate non proliferative retinopathy，laser spot', 'hypertensive retinopathy', 'post laser photocoagulation，glaucoma', 'lens dust，spotted membranous change', 'macular epiretinal membrane，mild nonproliferative retinopathy', 'macular hole', 'epiretinal membrane，moderate non proliferative retinopathy，laser spot', 'wet age-related macular degeneration', 'moderate non proliferative retinopathy，myelinated nerve fibers', 'dry age-related macular degeneration', 'lens dust，normal fundus', 'epiretinal membrane，myelinated nerve fibers', 'diabetic retinopathy', 'epiretinal membrane，lens dust', 'atrophy', 'laser spot，white ves

In [21]:
len(unique_right_labels)

233